<a href="https://colab.research.google.com/github/Madhuvod/vehicle-detection-minor/blob/main/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil

# Define the source and destination directories
source_dir = '/content/indian_images'
destination_dir = '/path/to/consolidated_dataset/'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Traverse all the subfolders and copy images to the destination folder
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.endswith('.jpg'):
            source_file = os.path.join(root, file)
            destination_file = os.path.join(destination_dir, file)
            shutil.copy(source_file, destination_file)

print(f'All images have been copied to {destination_dir}')


In [ ]:
# Install ultralytics package
!pip install -q ultralytics
!pip install  -q shapely

# Check if GPU is available
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.6/877.6 kB 6.5 MB/s eta 0:00:00


In [ ]:
import os
import json
from shapely.geometry import Polygon
from ultralytics import YOLO
import shutil



In [ ]:
# Define dataset paths
dataset_path = '/content/dataset'
images_path = os.path.join(dataset_path, 'images')
labels_path = os.path.join(dataset_path, 'labels')

# Create directories
os.makedirs(os.path.join(images_path, 'train'), exist_ok=True)
os.makedirs(os.path.join(images_path, 'val'), exist_ok=True)
os.makedirs(os.path.join(labels_path, 'train'), exist_ok=True)
os.makedirs(os.path.join(labels_path, 'val'), exist_ok=True)


In [ ]:
from google.colab import files

# Upload training images
print("Upload your 4 training images")
uploaded_train_images = files.upload()
for filename in uploaded_train_images.keys():
    shutil.move(filename, os.path.join(images_path, 'train', filename))

# Upload validation images
print("Upload your 2 validation images")
uploaded_val_images = files.upload()
for filename in uploaded_val_images.keys():
    shutil.move(filename, os.path.join(images_path, 'val', filename))

# Upload training labels
print("Upload your 4 training label files")
uploaded_train_labels = files.upload()
for filename in uploaded_train_labels.keys():
    shutil.move(filename, os.path.join(labels_path, 'train', filename))

# Upload validation labels
print("Upload your 2 validation label files")
uploaded_val_labels = files.upload()
for filename in uploaded_val_labels.keys():
    shutil.move(filename, os.path.join(labels_path, 'val', filename))


In [ ]:
# List training images
print("Training images:", os.listdir(os.path.join(images_path, 'train')))
# List validation images
print("Validation images:", os.listdir(os.path.join(images_path, 'val')))
# List training labels
print("Training labels:", os.listdir(os.path.join(labels_path, 'train')))
# List validation labels
print("Validation labels:", os.listdir(os.path.join(labels_path, 'val')))


In [ ]:
class_name_to_id = {
    'road': 0,
    'car': 1,
    'person': 2,
    'truck': 3,
    'bus': 4,
    'motorcycle': 5,
}

In [ ]:
# Define class names
class_names = ['road', 'car','person', 'truck', 'bus', 'motorcycle']

# Create the YAML content
data_yaml = f"""
train: {images_path}/train
val: {images_path}/val

nc: {len(class_names)}
names: {class_names}
"""

# Save the YAML file
with open('data.yaml', 'w') as f:
    f.write(data_yaml)

# Display the YAML file content
print(data_yaml)


In [ ]:
def convert_json_to_yolo(json_file_path, label_output_path, image_width, image_height):
    with open(json_file_path, 'r') as f:
        data = json.load(f)

    yolo_annotations = []

    for obj in data['objects']:
        label = obj['label']
        if label not in class_name_to_id:
            continue  # Skip classes not in our mapping

        class_id = class_name_to_id[label]
        polygon_points = obj['polygon']

        # Create a polygon
        poly = Polygon(polygon_points)
        if not poly.is_valid:
            poly = poly.buffer(0)

        # Get bounding box from polygon
        min_x, min_y, max_x, max_y = poly.bounds

        # Normalize coordinates
        x_center = ((min_x + max_x) / 2) / image_width
        y_center = ((min_y + max_y) / 2) / image_height
        width = (max_x - min_x) / image_width
        height = (max_y - min_y) / image_height

        # Ensure values are between 0 and 1
        x_center = min(max(x_center, 0), 1)
        y_center = min(max(y_center, 0), 1)
        width = min(max(width, 0), 1)
        height = min(max(height, 0), 1)

        yolo_annotations.append(f"{class_id} {x_center} {y_center} {width} {height}")

    # Save annotations to .txt file
    with open(label_output_path, 'w') as f:
        f.write('\n'.join(yolo_annotations))



In [ ]:
# Process each JSON file
labels_dir = '/content/dataset/labels/train'
for json_filename in os.listdir(labels_dir):
    if json_filename.endswith('.json'):
        json_file_path = os.path.join(labels_dir, json_filename)

        # Read JSON file
        with open(json_file_path, 'r') as f:
            data = json.load(f)

        # Get image dimensions
        img_width = data['imgWidth']
        img_height = data['imgHeight']

        # Prepare paths
        image_base_name = os.path.splitext(json_filename)[0]
        label_output_path = os.path.join(labels_dir, image_base_name + '.txt')

        # Convert JSON to YOLO format
        convert_json_to_yolo(json_file_path, label_output_path, img_width, img_height)


In [ ]:
# List the generated label files
print("Generated label files:")
print(os.listdir(labels_dir))

# Optionally, print contents of a label file
with open(os.path.join(labels_dir, '/content/dataset/labels/train/frame12335_gtFine_polygons.json'), 'r') as f:
    print(f.read())


In [ ]:
# Define class names (order must match class IDs)
class_names = ['road', 'car', 'person', 'truck', 'bus', 'motorcycle']
dataset_path = '/content/dataset'
# Create the YAML content
data_yaml = f"""
path: {dataset_path}

train: /content/dataset/images/train
val: /content/dataset/images/val

nc: {len(class_names)}
names: {class_names}
"""

# Save the YAML file
with open('data.yaml', 'w') as f:
    f.write(data_yaml)

print(data_yaml)


In [ ]:
# Initialize the model
model = YOLO('yolov8n.pt')

# Train the model
model.train(
    data='data.yaml',
    epochs=10,
    imgsz=640,
    batch=2,
    name='yolov8_custom',
    verbose=True
)



In [ ]:
# Validate the model
metrics = model.val()

# Test the model on a validation image
test_image_path = os.path.join(images_dir, 'val', val_images[0])  # Use the first validation image
results = model.predict(source=test_image_path, save=True, conf=0.25)

from IPython.display import Image
display(Image(filename='runs/detect/predict/image0.jpg'))
